In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from glob import glob

In [79]:
os.environ["PHTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [80]:
batch_size = 4
lr = 1e-4
epochs = 4
height = 752
width = 432

In [81]:
drive_path = "/content/drive/MyDrive/"

In [82]:
dataset_path = os.path.join(drive_path,"Cancer Dataset")

files_dir = os.path.join(drive_path ,"Colab Notebooks","files2","non-aug2")
model_file = os.path.join(files_dir,"Unet-non-aug.h5")
log_file = os.path.join(files_dir,"Log-non-aug.csv")

### Creating folder

In [83]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [84]:
create_dir(files_dir)

### Building UNET

#### Conv Block

In [85]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

#### Encoder Block

In [86]:
from tensorflow.keras.layers import MaxPooling2D

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPooling2D((2, 2))(x)
    return x,p

#### Decoder lock

In [87]:
from tensorflow.keras.layers import Concatenate, Conv2DTranspose


def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)

    skip = tf.image.resize(skip, (x.shape[1], x.shape[2]))

    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

#### UNET

In [88]:
from keras.layers import Input
from keras.layers import Conv2D
import tensorflow as tf

def build_unet(input_shape):
    inputs = Input(shape=input_shape)

    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """Bridge"""
    b1 = conv_block(p4, 1024)

    """Decoder"""
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = tf.keras.Model(inputs, outputs, name="UNET")

    return model

In [89]:
import os
from glob import glob

def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train","adenocarcinoma", "*")))
    train_y = sorted(glob(os.path.join(path,"train","mask", "*")))
    valid_x = sorted(glob(os.path.join(path, "valid","adenocarcinoma", "*")))
    valid_y = sorted(glob(os.path.join(path,"valid","mask", "*")))

    return (train_x, train_y), (valid_x, valid_y)

#### Reading Images

In [90]:
def read_image(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = x/255.0

  return x

In [91]:
def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = x/255.0
  x = np.expand_dims(x, axis=-1)
  return x

#### tf.data pipeline

In [92]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)

        x = tf.image.resize(x, [height, width])
        y = tf.image.resize(y, [height, width])

        x = tf.dtypes.cast(x, tf.float64)
        y = tf.dtypes.cast(y, tf.float64)

        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])

    return x, y


In [93]:
def tf_dataset(x, y, batch=1):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [94]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)

In [95]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 101 - 101
Valid: 18 - 18


In [96]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [97]:
for x,y in valid_dataset:
  print(x.shape, y.shape)

(4, 752, 432, 3) (4, 752, 432, 1)
(4, 752, 432, 3) (4, 752, 432, 1)
(4, 752, 432, 3) (4, 752, 432, 1)
(4, 752, 432, 3) (4, 752, 432, 1)
(2, 752, 432, 3) (2, 752, 432, 1)


In [98]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [99]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 752, 432, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 752, 432, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 752, 432, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 752, 432, 64)         0         ['batch_normalization_18[0]

In [100]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=["acc"])

In [101]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
]


In [102]:
model.fit(
    train_dataset,
    validation_data = valid_dataset,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/4
26/26 [==============================] - ETA: 0s - loss: 0.6728 - acc: 0.6434
Epoch 1: val_loss improved from inf to 0.79220, saving model to /content/drive/MyDrive/Colab Notebooks/files2/non-aug2/Unet-non-aug.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 79s 3s/step - loss: 0.6728 - acc: 0.6434 - val_loss: 0.7922 - val_acc: 0.0728 - lr: 1.0000e-04
Epoch 2/4
26/26 [==============================] - ETA: 0s - loss: 0.2754 - acc: 0.9796
Epoch 2: val_loss improved from 0.79220 to 0.59781, saving model to /content/drive/MyDrive/Colab Notebooks/files2/non-aug2/Unet-non-aug.h5
26/26 [==============================] - 49s 2s/step - loss: 0.2754 - acc: 0.9796 - val_loss: 0.5978 - val_acc: 0.9806 - lr: 1.0000e-04
Epoch 3/4
26/26 [==============================] - ETA: 0s - loss: 0.2219 - acc: 0.9818
Epoch 3: val_loss improved from 0.59781 to 0.54638, saving model to /content/drive/MyDrive/Colab Notebooks/files2/non-aug2/Unet-non-aug.h5
26/26 [==============================] - 45s 2s/step - loss: 0.2219 - acc: 0.9818 - val_loss: 0.5464 - val_acc: 0.9540 - lr: 1.0000e-04
Epoch 4/4
26/26 [==============================] - ETA: 0s - loss: 0.1865 - acc: 0.9836
Epoch 4: val_loss improved from 0.54638 to 0.48838

### Predicting the Masks

In [103]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

### Seeding

In [104]:
os.environ["PHTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [105]:
height = 752
width = 432

### Test Path

In [106]:
dataset_path_test = "/content/drive/MyDrive/Cancer Dataset/test2"
save_path = "/content/drive/MyDrive/Cancer Dataset/ prediction/non-aug"
model_path = "/content/drive/MyDrive/Colab Notebooks/files2/non-aug2/Unet-non-aug.h5"

### Folder Prediction

In [107]:
create_dir(save_path)

### Loading Model

In [108]:
model = tf.keras.models.load_model(model_path)

In [109]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 752, 432, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 752, 432, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 752, 432, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 752, 432, 64)         0         ['batch_normalization_18[0]

### Load test dataset

In [110]:
test_x = sorted(glob(os.path.join(dataset_path_test,"test_image","*" )))
print(f"Test Images:{len(test_x)}")

Test Images:67


In [111]:
test_x[0]

'/content/drive/MyDrive/Cancer Dataset/test2/test_image/1.png'

In [112]:
import os
import cv2


input_folder = '/content/drive/MyDrive/Cancer Dataset/test/adenocarcinoma_test'
output_folder = '/content/drive/MyDrive/Cancer Dataset/test2/test_image'


target_size = (432, 752)


if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for filename in os.listdir(input_folder):

    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):

        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)


        resized_image = cv2.resize(image, target_size)


        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_image)

print("Resizing complete.")


Resizing complete.


In [113]:
# Mask resizing
import os
import cv2


input_folder = '/content/drive/MyDrive/Cancer Dataset/test/mask'
output_folder = '/content/drive/MyDrive/Cancer Dataset/test2/test_mask'


target_size = (432, 752)


if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        resized_image = cv2.resize(image, target_size)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_image)

print("Resizing complete.")


Resizing complete.


### Predict th mask

In [114]:
time_taken = []

for x in tqdm(test_x):
  print("")
  name = x.split("/")[-1]

  x = cv2.imread(x, cv2.IMREAD_COLOR)
  x = x / 255.0
  x = np.expand_dims(x, axis = 0)

  start_time = time.time()
  p = model.predict(x)[0]
  total_time = time.time() - start_time
  time_taken.append(total_time)

  p = p*255
  cv2.imwrite(os.path.join(save_path, name), p)

  0%|          | 0/67 [00:00<?, ?it/s]


1/1 [==============================] - 0s 429ms/step


  1%|▏         | 1/67 [00:00<00:44,  1.49it/s]


1/1 [==============================] - 0s 21ms/step


  3%|▎         | 2/67 [00:00<00:26,  2.44it/s]

...

 97%|█████████▋| 65/67 [00:14<00:00,  5.18it/s]


1/1 [==============================] - 0s 21ms/step


 99%|█████████▊| 66/67 [00:15<00:00,  4.96it/s]


1/1 [==============================] - 0s 23ms/step


100%|██████████| 67/67 [00:15<00:00,  4.36it/s]


### Calculating Accuracy

In [115]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

In [116]:
pred_mask_data = "/content/drive/MyDrive/Cancer_Dataset/prediction"

In [117]:
pred_mask = sorted(glob(os.path.join("/content/drive/MyDrive/Cancer Dataset/ prediction/non-aug/*")))
true_mask = sorted(glob(os.path.join("/content/drive/MyDrive/Cancer Dataset/test2/test_mask/*")))

In [118]:
print(pred_mask[0], true_mask[0])

/content/drive/MyDrive/Cancer Dataset/ prediction/non-aug/1.png /content/drive/MyDrive/Cancer Dataset/test2/test_mask/1.png


In [119]:
score = []

for pred_y, true_y in tqdm(zip(pred_mask, true_mask), total=len(pred_mask)):
  name = pred_y.split("/")[-1]

  pred_y = cv2.imread(pred_y, cv2.IMREAD_GRAYSCALE)
  pred_y = pred_y/255.0
  pred_y = pred_y > 0.5
  pred_y = pred_y.astype(np.int32)
  pred_y = pred_y.flatten()

  true_y = cv2.imread(true_y, cv2.IMREAD_GRAYSCALE)
  true_y = true_y/255.0
  true_y = true_y > 0.5
  true_y = true_y.astype(np.int32)
  true_y = true_y.flatten()

  acc_value = accuracy_score(pred_y, true_y)
  ft_value = f1_score(pred_y, true_y, labels=[0,1], average="binary")
  score.append([name, acc_value])


100%|██████████| 67/67 [00:10<00:00,  6.27it/s]


In [120]:
score = [s[1:] for s in score]

In [121]:
mean_score = np.mean(score, axis=0)

In [122]:
print(f"Accuracy: {mean_score[0]:0.5f}")

Accuracy: 0.98498
